In [1]:
from __future__ import print_function

from miscc.utils import mkdir_p
from miscc.utils import build_super_images
from miscc.losses import sent_loss, words_loss
from miscc.config import cfg, cfg_from_file
from DAMSM_train import train, evaluate, build_models, parse_args
from datasets import TextDataset
from datasets import prepare_data

from model import RNN_ENCODER, CNN_ENCODER

import glob
import os
import sys
# 파일 경로 조작 및 시스템 경로 설정
import time
# 코드 실행 시간을 측정
import random
# 난수 생성
import pprint
# 설정 파일을 예쁘게 출력하는 용도
import datetime
import dateutil.tz
# 현재 시간과 타임스탬프를 관리
import argparse
import numpy as np
# 배열 연산을 위해 사용
from PIL import Image
# 이미지 저장 및 변환

import torch
import torch.nn as nn
# 기본적인 PyTorch 기능과 신경망 모듈
import torch.optim as optim
# 옵티마이저 관련 모듈
from torch.autograd import Variable
# 자동 미분 기능을 위한 래퍼 (PyTorch 최신 버전에서는 필요 X)
import torch.backends.cudnn as cudnn
# GPU 연산 최적화
import torchvision.transforms as transforms
# 이미지 전처리(transform) 관련 기능 제공

In [2]:
dir_path = os.path.abspath(os.path.join(os.getcwd(), './.'))
# .../image_processing/code
sys.path.append(dir_path)

print(dir_path)

c:\Users\blues\바탕 화면\Hyewon\Study\SIG\2024-SIG\2024_winter_sig\image_processing\code


In [3]:
# sys.argv = ['pretrain_DAMSM.py', '--cfg', 'cfg/DAMSM/coco.yml', '--gpu', '0'] 맨 앞의 parameter 어떻게 설정해두는 게 맞는 지 확인하기기
sys.argv = ['DAMSM_train.py', '--cfg', 'cfg/DAMSM/coco.yml', '--gpu', '0']

args = parse_args()
print(args)

Namespace(cfg_file='cfg/DAMSM/coco.yml', gpu_id=0, data_dir='', manualSeed=None)


In [4]:
if args.cfg_file is not None:
     cfg_from_file(args.cfg_file)

if args.gpu_id == -1:
    cfg.CUDA = False
else:
    cfg.GPU_ID = args.gpu_id

if args.data_dir != '': # args.data_dir = ../data/coco
    cfg.DATA_DIR = args.data_dir
    # DATA_DIR: '../data/coco'
print('Using config:')
pprint.pprint(cfg)


Using config:
{'B_VALIDATION': False,
 'CONFIG_NAME': 'DAMSM',
 'CUDA': True,
 'DATASET_NAME': 'coco',
 'DATA_DIR': '../data/coco',
 'GAN': {'B_ATTENTION': True,
         'B_DCGAN': False,
         'CONDITION_DIM': 100,
         'DF_DIM': 64,
         'GF_DIM': 128,
         'R_NUM': 2,
         'Z_DIM': 100},
 'GPU_ID': 0,
 'RNN_TYPE': 'LSTM',
 'TEXT': {'CAPTIONS_PER_IMAGE': 5, 'EMBEDDING_DIM': 256, 'WORDS_NUM': 15},
 'TRAIN': {'BATCH_SIZE': 48,
           'B_NET_D': True,
           'DISCRIMINATOR_LR': 0.0002,
           'ENCODER_LR': 0.002,
           'FLAG': True,
           'GENERATOR_LR': 0.0002,
           'MAX_EPOCH': 200,
           'NET_E': '',
           'NET_G': '',
           'RNN_GRAD_CLIP': 0.25,
           'SMOOTH': {'GAMMA1': 4.0,
                      'GAMMA2': 5.0,
                      'GAMMA3': 10.0,
                      'LAMBDA': 1.0},
           'SNAPSHOT_INTERVAL': 5},
 'TREE': {'BASE_SIZE': 299, 'BRANCH_NUM': 1},
 'WORKERS': 1}


In [5]:

print('manualSeed:', args.manualSeed)  # manualSeed 값 출력

if not cfg.TRAIN.FLAG:
    args.manualSeed = 100
# coco.yml에서는 cfg.TRAIN.FLAG = true
elif args.manualSeed is None:
    args.manualSeed = random.randint(1, 10000)
    # 1 ~ 10000 사이로 랜덤 값으로 설정정
    
print('Updated manualSeed:', args.manualSeed)

# seed 값 설정해주어서 동일한 난수를 생성하도록록
random.seed(args.manualSeed)
np.random.seed(args.manualSeed)
torch.manual_seed(args.manualSeed)
if cfg.CUDA: # CUDA 사용하면 해당 GPU에서도 동일한 난수 생성하도록 설정정
    torch.cuda.manual_seed_all(args.manualSeed)

manualSeed: None
Updated manualSeed: 1616


In [5]:
now = datetime.datetime.now(dateutil.tz.tzlocal())
timestamp = now.strftime('%Y_%m_%d_%H_%M_%S')
# dir_path : c:\Users\blues\바탕 화면\혜원\개인공부\SIG\2024-SIG\2024_winter_sig\image_processing\code

output_dir = '../output/%s_%s_%s' % \
    (cfg.DATASET_NAME, cfg.CONFIG_NAME, timestamp) 
# 디렉토리 경로 생성

model_dir = os.path.join(output_dir, 'Model')
# 여기서 저장된 모델을 어떻게 사용할건지에 대해서 코드보고 이해하기

vocadict_dir = os.path.join(cfg.DATA_DIR, 'voca_dictionary')
# 단어 사전은 data/coco/vaca_Dictionary으로 해서 중복으로 단어 사전 생성 안하도록 하기

image_dir = os.path.join(output_dir, 'Image')
# 모델과 이미지 저장용 디렉토리 경로 설정

mkdir_p(model_dir)
# mkdir_p : 지정된 디렉토리가 존재하지 않으면 생성하는 함수
mkdir_p(vocadict_dir)
mkdir_p(image_dir)

torch.cuda.set_device(cfg.GPU_ID)
# 저장된 GPU ID로 GPU를 설정한다.
cudnn.benchmark = True
# cuDNN의 최적화 활성화화

In [5]:
vocadict_dir = os.path.join(cfg.DATA_DIR, 'voca_dictionary')

imsize = cfg.TREE.BASE_SIZE * (2 ** (cfg.TREE.BRANCH_NUM-1))
# 입력 이미지의 크기를 결정
batch_size = cfg.TRAIN.BATCH_SIZE

## 이미지 전처리
image_transform = transforms.Compose([
    transforms.Resize(int(imsize * 76 / 64)),
    transforms.RandomCrop(imsize),
    transforms.RandomHorizontalFlip()])

#### train dataset Load
dataset_train = TextDataset(cfg.DATA_DIR, vocadict_dir, 'train',
                          base_size=cfg.TREE.BASE_SIZE,
                          transform=image_transform)
# vocadict_dir = cfg.DATA_DIR\voca_dictionary
print(dataset_train)  # None이면 데이터셋이 생성되지 않은 것
print(dataset_train.n_words, dataset_train.embeddings_num)
"""
dataset.nwords : 데이터 셋의 단어 수
dataset.embeddings_num : 임베딩 크기 
"""
assert dataset_train, "Invalid train Dataset"
# PyThon에서 단순한 존재 검증 즉, True로 평가되는 값인지 확인
# None 값, 빈 리스트, 빈 문자열, 0, False 같은 값이면 AssertionError 발생

# DataLoader : PyTorch에서 데이터를 배치 단위로 처리하기 위한 도구
dataloader_train = torch.utils.data.DataLoader(
    dataset_train, batch_size=batch_size, drop_last=True,
    shuffle=True, num_workers=int(cfg.WORKERS))
# drop_last=True : 데이터셋의 크기가 배치 크기로 나누어떨어지지 않는 경우 마지막 배치를 버린다.

#### validation dataset Load
dataset_val = TextDataset(cfg.DATA_DIR, vocadict_dir, 'val',
                        base_size=cfg.TREE.BASE_SIZE,
                        transform=image_transform)
print(dataset_val)
print(dataset_val.n_words, dataset_val.embeddings_num)

assert dataset_val, "Invalid val Dataset"

dataloader_val = torch.utils.data.DataLoader(
    dataset_val, batch_size=batch_size, drop_last=True,
    shuffle=True, num_workers=int(cfg.WORKERS))


Load pickle file from:  ../data/coco\voca_dictionary\captions.pickle
27312 5
Load pickle file from:  ../data/coco\voca_dictionary\captions.pickle
27312 5


In [7]:
text_encoder, image_encoder, labels, start_epoch = build_models(dataset_train, batch_size)
para = list(text_encoder.parameters())
# text_encoder의 모든 학습 가능 파라미터를 리스트에 추가
for v in image_encoder.parameters():
    if v.requires_grad:
        # requires_grad=False인 경우 역전파에서 제외시킴킴
        para.append(v)
# optimizer = optim.Adam(para, lr=cfg.TRAIN.ENCODER_LR, betas=(0.5, 0.999))
# At any point you can hit Ctrl + C to break out of training early.

# 가장 낮은 검증 손실을 저장
best_val_loss = float('inf')

# 옵티마이저를 한 번만 생성
optimizer = optim.Adam(para, lr=cfg.TRAIN.ENCODER_LR, betas=(0.5, 0.999))
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.98)

try:
    lr = cfg.TRAIN.ENCODER_LR # 초기 학습률(lr) 설정
    for epoch in range(start_epoch, cfg.TRAIN.MAX_EPOCH):
        # start_epoch부터 MAX_EPOCH까지 학습 반복
        # optimizer = optim.Adam(para, lr=lr, betas=(0.5, 0.999))
        # Adam 옵티마이저를 사용하여 text/image_encoder를 학습
        # 일반적으로는 betas=(0.9, 0.999)를 이용

        epoch_start_time = time.time()
        count = train(dataloader_train, image_encoder, text_encoder,
                    batch_size, labels, optimizer, epoch,
                    dataset_train.ixtoword, image_dir)
        # 모델을 한 epoch 동안 학습하는 역할을 함
        print('-' * 89)

        if len(dataloader_val) > 0:
            # 학습한 모델을 검증 데이터에서 평가
            # 가중치를 업데이트하지 않고, 현재 모델이 얼마나 잘 동작하는지 평가가
            s_loss, w_loss = evaluate(dataloader_val, image_encoder,
                                      text_encoder, batch_size, labels)
            val_loss = s_loss + w_loss # 검증 손실 합산
            print('| end epoch {:3d} | valid loss {:5.2f} {:5.2f} | lr {:.5f}|'
                  .format(epoch, s_loss, w_loss, scheduler.get_last_lr()[0]))
            
            #### 가장 낮은 검증 손실을 가진 모델 저장
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                torch.save(image_encoder.state_dict(),
                           f'{model_dir}/best_image_encoder.pth')
                torch.save(text_encoder.state_dict(),
                           f'{model_dir}/best_text_encoder.pth')
                print('Best model saved!')
            
        print('-' * 89)
        
        scheduler.step() # 스케줄러로 학습률 감소

        #if lr > cfg.TRAIN.ENCODER_LR/10.:
            # 현재 학습률이 초기 학습률의 1/10 이상이면 lr를 98%로 감소시킴
        #    lr *= 0.98

        # 모델 저장
        if (epoch % cfg.TRAIN.SNAPSHOT_INTERVAL == 0 or
            epoch == cfg.TRAIN.MAX_EPOCH):
            # 일정 epoch마다 또는 MAX_EPOCH이 되면 가중치 저장
            torch.save(image_encoder.state_dict(),
                       '%s/image_encoder%d.pth' % (model_dir, epoch))
            torch.save(text_encoder.state_dict(),
                       '%s/text_encoder%d.pth' % (model_dir, epoch))
            print('Save models.')
            
except KeyboardInterrupt:
    print('-' * 89)
    print('Exiting from training early')

c:\Users\blues\anaconda3\envs\2024WINTERSIG\Lib\site-packages\torch\nn\modules\rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn(
c:\Users\blues\anaconda3\envs\2024WINTERSIG\Lib\site-packages\torchvision\models\inception.py:43: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn(


Load pretrained model from  https://download.pytorch.org/models/inception_v3_google-1a9a5a14.pth


c:\Users\blues\anaconda3\envs\2024WINTERSIG\Lib\site-packages\torch\nn\modules\module.py:1736: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


| epoch   0 |     0/ 2464 batches | ms/batch 72.28 | s_loss  0.02  0.02 | w_loss  0.03  0.02


c:\Users\blues\anaconda3\envs\2024WINTERSIG\Lib\site-packages\torch\nn\modules\module.py:1736: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


| epoch   0 |   200/ 2464 batches | ms/batch 1522.12 | s_loss  2.64  2.64 | w_loss  2.82  2.63
| epoch   0 |   400/ 2464 batches | ms/batch 1449.57 | s_loss  1.64  1.64 | w_loss  1.68  1.61
| epoch   0 |   600/ 2464 batches | ms/batch 1420.95 | s_loss  1.38  1.39 | w_loss  1.39  1.35
| epoch   0 |   800/ 2464 batches | ms/batch 1449.84 | s_loss  1.27  1.28 | w_loss  1.26  1.22
| epoch   0 |  1000/ 2464 batches | ms/batch 1447.58 | s_loss  1.16  1.18 | w_loss  1.15  1.15
| epoch   0 |  1200/ 2464 batches | ms/batch 1429.11 | s_loss  1.11  1.13 | w_loss  1.08  1.08
| epoch   0 |  1400/ 2464 batches | ms/batch 1450.60 | s_loss  1.08  1.11 | w_loss  1.05  1.06
| epoch   0 |  1600/ 2464 batches | ms/batch 1459.43 | s_loss  1.03  1.05 | w_loss  1.00  1.00
| epoch   0 |  1800/ 2464 batches | ms/batch 1462.70 | s_loss  1.01  1.03 | w_loss  0.98  0.98
| epoch   0 |  2000/ 2464 batches | ms/batch 1452.44 | s_loss  0.98  1.01 | w_loss  0.92  0.94
| epoch   0 |  2200/ 2464 batches | ms/batch 1453.

In [15]:
from torch.amp import GradScaler
import re
UPDATE_INTERVAL = 200
from torch.amp import autocast

def next_train(dataloader, cnn_model, rnn_model, batch_size,
          labels, optimizer, epoch, ixtoword, image_dir, scaler):
    # dataloader : 데이터셋을 불러오는 DataLoader 객체
    # labels : 배치 크기만큼의 정수 라벨 텐서 (0 ~ batch_size -1)
    # ixtoword : 인덱스를 단어로 변환하는 딕셔너리
    # image__dir : 학습 중 생성된 attentio map 이미지를 저장할 디렉토리

    cnn_model.train() # 모델이 학습 모드로 설정됨 → Dropout과 같이 학습 중에만 활성화되는 연산이 실행됨됨
    rnn_model.train()
    s_total_loss0 = 0
    s_total_loss1 = 0
    w_total_loss0 = 0
    w_total_loss1 = 0
    count = (epoch + 1) * len(dataloader) # 학습된 데이터 샘플 개수를 추적
    start_time = time.time() # 학습 속도를 측정하기 위한 시작 시간 저장
    for step, data in enumerate(dataloader, 0):
        # dataloader에서 데이터를 하나씩 꺼내 학습을 진행

        # print('step', step)
        # 이전 step의 그래디언트 값을 초기화화
        rnn_model.zero_grad()
        cnn_model.zero_grad()

        imgs, captions, cap_lens, class_ids, keys = prepare_data(data)
        # prepare_data를 이용하여 데이터 정렬

        with autocast(device_type='cuda'):
            words_features, sent_code = cnn_model(imgs[-1]) # CNN 인코더 적용
            nef, att_sze = words_features.size(1), words_features.size(2)
            hidden = rnn_model.init_hidden(batch_size) # RNN 인코더 적용
            words_emb, sent_emb = rnn_model(captions, cap_lens, hidden)

            w_loss0, w_loss1, attn_maps = words_loss(words_features, words_emb, labels,
                                                 cap_lens, class_ids, batch_size)
            w_total_loss0 += w_loss0.item()
            w_total_loss1 += w_loss1.item()
            loss = w_loss0 + w_loss1

            s_loss0, s_loss1 = \
            sent_loss(sent_code, sent_emb, labels, class_ids, batch_size)
            loss += s_loss0 + s_loss1
            s_total_loss0 += s_loss0.item()
            s_total_loss1 += s_loss1.item()

        scaler.scale(loss).backward()  # loss에 대해 그래디언트를 계산하고 스케일링
        torch.nn.utils.clip_grad_norm_(rnn_model.parameters(), cfg.TRAIN.RNN_GRAD_CLIP)
        
            # optimizer.step() 전에 scaler.step()을 호출하여 가중치를 업데이트
        scaler.step(optimizer)  # 가중치 업데이트
        scaler.update()  # 스케일러 업데이트

        if step % UPDATE_INTERVAL == 0:
                # 일정한 간격마다 손실값 및 학습 속도 출력
            count = epoch * len(dataloader) + step

            s_cur_loss0 = s_total_loss0 / UPDATE_INTERVAL
            s_cur_loss1 = s_total_loss1 / UPDATE_INTERVAL

            w_cur_loss0 = w_total_loss0 / UPDATE_INTERVAL
            w_cur_loss1 = w_total_loss1 / UPDATE_INTERVAL

            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches | ms/batch {:5.2f} | '
                    's_loss {:5.2f} {:5.2f} | '
                    'w_loss {:5.2f} {:5.2f}'
                    .format(epoch, step, len(dataloader),
                            elapsed * 1000. / UPDATE_INTERVAL,
                            s_cur_loss0, s_cur_loss1,
                            w_cur_loss0, w_cur_loss1))
            s_total_loss0 = 0
            s_total_loss1 = 0
            w_total_loss0 = 0
            w_total_loss1 = 0
            start_time = time.time()
                # attention Maps
            img_set, _ = \
                    build_super_images(imgs[-1].cpu(), captions, ixtoword, attn_maps, att_sze)
            """
            build_super_images : 실제 이미지, 어텐션 맵, 캡션을 기반으로 시각적 주의를 시각화한 이미지를 생성하는 역할
            텍스트 설명과 함께 이미지와 attention map을 한 장의 이미지로 구성.
            img_set : attention map이 적용된 이미지가 저장됨
            _ : 두 번째 반환값이 있지만 사용되지 않음을 의미
            imgs[-1].cpu() : 마지막 단계의 이미지 텐서를 CPU로 이동시킴

            """

            # 이미지가 유효한 경우 실행
            if img_set is not None:
                im = Image.fromarray(img_set)
                # Numpy 배열을 PIL 이미지 객체로 변환
                image_dir_per_epoch = os.path.join(image_dir, str(epoch))
                mkdir_p(image_dir_per_epoch)
                fullpath = '%s/attention_maps_step%d.png' % (image_dir_per_epoch, step)
                # 저장 경로 설정, step : 현재 학습 단계(step) 번호
                im.save(fullpath)
    return count


def next_evaluate(dataloader, cnn_model, rnn_model, batch_size, labels):
    cnn_model.eval() # 모델이 평가 모드로 설정됨
    rnn_model.eval()
    s_total_loss = 0
    w_total_loss = 0

    with autocast(device_type='cuda'):
        for step, data in enumerate(dataloader, 0):
            real_imgs, captions, cap_lens, class_ids, keys = prepare_data(data)

            words_features, sent_code = cnn_model(real_imgs[-1])
            # nef = words_features.size(1)
            # words_features = words_features.view(batch_size, nef, -1)

            hidden = rnn_model.init_hidden(batch_size)
            words_emb, sent_emb = rnn_model(captions, cap_lens, hidden)

            w_loss0, w_loss1, attn = words_loss(words_features, words_emb, labels,
                                                cap_lens, class_ids, batch_size)
            w_total_loss += (w_loss0 + w_loss1).data

            s_loss0, s_loss1 = \
                sent_loss(sent_code, sent_emb, labels, class_ids, batch_size)
            s_total_loss += (s_loss0 + s_loss1).data

            if step == 50:
                break

    s_cur_loss = s_total_loss.item() / step
    w_cur_loss = w_total_loss.item() / step

    return s_cur_loss, w_cur_loss

def build_next_models(dataset, batch_size):
    # build model ############################################################
    text_encoder = RNN_ENCODER(dataset.n_words, nhidden=cfg.TEXT.EMBEDDING_DIM)
    # RNN_ENCODER : 텍스트 데이터를 인코딩하는 RNN 모델
    # n_words : 단어 크기 (텍스트에 포함된 전체 단어
    # cfg.TEXT.EMBEDDING_DIM = 256, RNN의 은닉 상태 크기

    image_encoder = CNN_ENCODER(cfg.TEXT.EMBEDDING_DIM)
    # CNN_ENCODER : 이미지를 인코딩하는 CNN 모델
    # 텍스 임베딩 차원을 입력으로 받음 → 나중에 텍스트와 이미지 특징을 비교하기 위해해

    # labels = Variable(torch.LongTensor(range(batch_size)))
    labels = torch.arange(batch_size, dtype=torch.long)
    # 배치 크기만큼의 정수 라벨 텐서를 생성 : [0, 1, 2, ..., batch_size -1]
    # 모델 학습 시, 배치 내 각 샘플의 인덱스를 나타내는 역할

    start_epoch = 0
    model_path = '../output/coco_DAMSM_2025_02_19_00_04_58/Model/text_encoder10.pth'
    state_dict = torch.load(model_path)
    text_encoder.load_state_dict(state_dict)
    print('Load ', model_path)
    

    name = '../output/coco_DAMSM_2025_02_19_00_04_58/Model/image_encoder10.pth'
    state_dict = torch.load(name)
    image_encoder.load_state_dict(state_dict)
    print('Load ', name)
    
    model_filename = model_path.split('/')[-1]
    epoch_str = re.search(r'\d+', model_filename).group()  # 첫 번째 숫자 추출
    start_epoch = int(epoch_str) + 1  # 다음 epoch 번호

    print('start_epoch', start_epoch)

    if cfg.CUDA:
        # cfg.CUDA == True라면, GPU에서 모델을 실행하도록 .cuda()를 호출
        text_encoder = text_encoder.cuda()
        image_encoder = image_encoder.cuda()
        labels = labels.cuda() # 배치별 정수 라벨 텐서
        
    return text_encoder, image_encoder, labels, start_epoch


text_encoder, image_encoder, labels, start_epoch = build_next_models(dataset_train, batch_size)
scaler = GradScaler()

para = list(text_encoder.parameters())
# text_encoder의 모든 학습 가능 파라미터를 리스트에 추가
for v in image_encoder.parameters():
    if v.requires_grad:
        # requires_grad=False인 경우 역전파에서 제외시킴
        para.append(v)
# optimizer = optim.Adam(para, lr=cfg.TRAIN.ENCODER_LR, betas=(0.5, 0.999))
# At any point you can hit Ctrl + C to break out of training early.

# 가장 낮은 검증 손실을 저장
best_val_loss = float('inf')

# 옵티마이저를 한 번만 생성
try:
    lr = cfg.TRAIN.ENCODER_LR # 초기 학습률(lr) 설정
    for epoch in range(start_epoch, cfg.TRAIN.MAX_EPOCH):
        # start_epoch부터 MAX_EPOCH까지 학습 반복
        # optimizer = optim.Adam(para, lr=lr, betas=(0.5, 0.999))
        # Adam 옵티마이저를 사용하여 text/image_encoder를 학습
        # 일반적으로는 betas=(0.9, 0.999)를 이용

        epoch_start_time = time.time()
        count = next_train(dataloader_train, image_encoder, text_encoder,
                    batch_size, labels, optimizer, epoch,
                    dataset_train.ixtoword, image_dir, scaler)
        # 모델을 한 epoch 동안 학습하는 역할을 함
        print('-' * 89)

        if len(dataloader_val) > 0:
            # 학습한 모델을 검증 데이터에서 평가
            # 가중치를 업데이트하지 않고, 현재 모델이 얼마나 잘 동작하는지 평가가
            s_loss, w_loss = next_evaluate(dataloader_val, image_encoder,
                                      text_encoder, batch_size, labels)
            val_loss = s_loss + w_loss # 검증 손실 합산
            print('| end epoch {:3d} | valid loss {:5.2f} {:5.2f} | lr {:.5f}|'
                  .format(epoch, s_loss, w_loss, scheduler.get_last_lr()[0]))
            
            #### 가장 낮은 검증 손실을 가진 모델 저장
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                torch.save(image_encoder.state_dict(),
                           f'{model_dir}/best_image_encoder.pth')
                torch.save(text_encoder.state_dict(),
                           f'{model_dir}/best_text_encoder.pth')
                print('Best model saved!')
            
        print('-' * 89)
        
        scheduler.step() # 스케줄러로 학습률 감소

        #if lr > cfg.TRAIN.ENCODER_LR/10.:
            # 현재 학습률이 초기 학습률의 1/10 이상이면 lr를 98%로 감소시킴
        #    lr *= 0.98

        # 모델 저장
        if (epoch % cfg.TRAIN.SNAPSHOT_INTERVAL == 0 or
            epoch == cfg.TRAIN.MAX_EPOCH):
            # 일정 epoch마다 또는 MAX_EPOCH이 되면 가중치 저장
            torch.save(image_encoder.state_dict(),
                       '%s/image_encoder%d.pth' % (model_dir, epoch))
            torch.save(text_encoder.state_dict(),
                       '%s/text_encoder%d.pth' % (model_dir, epoch))
            print('Save models.')
            
except KeyboardInterrupt:
    print('-' * 89)
    print('Exiting from training early')

Load pretrained model from  https://download.pytorch.org/models/inception_v3_google-1a9a5a14.pth
Load  ../output/coco_DAMSM_2025_02_19_00_04_58/Model/text_encoder10.pth


C:\Users\blues\AppData\Local\Temp\ipykernel_7512\2472341345.py:155: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(model_path)
C:\Users\blues\AppData\

Load  ../output/coco_DAMSM_2025_02_19_00_04_58/Model/image_encoder10.pth
start_epoch 11
| epoch  11 |     0/ 2464 batches | ms/batch 44.59 | s_loss  0.00  0.00 | w_loss  0.00  0.00
| epoch  11 |   200/ 2464 batches | ms/batch 1098.46 | s_loss  0.69  0.71 | w_loss  0.55  0.57
| epoch  11 |   400/ 2464 batches | ms/batch 1076.17 | s_loss  0.69  0.71 | w_loss  0.54  0.56
| epoch  11 |   600/ 2464 batches | ms/batch 1131.07 | s_loss  0.69  0.71 | w_loss  0.56  0.56
| epoch  11 |   800/ 2464 batches | ms/batch 1144.42 | s_loss  0.68  0.70 | w_loss  0.54  0.56
| epoch  11 |  1000/ 2464 batches | ms/batch 1216.68 | s_loss  0.69  0.72 | w_loss  0.54  0.56
| epoch  11 |  1200/ 2464 batches | ms/batch 1186.81 | s_loss  0.69  0.71 | w_loss  0.55  0.56
| epoch  11 |  1400/ 2464 batches | ms/batch 1262.33 | s_loss  0.69  0.71 | w_loss  0.54  0.56
| epoch  11 |  1600/ 2464 batches | ms/batch 1269.53 | s_loss  0.69  0.71 | w_loss  0.56  0.57
| epoch  11 |  1800/ 2464 batches | ms/batch 1144.60 | s_lo

In [10]:
from torch.amp import GradScaler
import re
UPDATE_INTERVAL = 200
from torch.amp import autocast
from datasets import TextDataset
from miscc.config import cfg, cfg_from_file
import torchvision.transforms as transforms



def next_train(dataloader, cnn_model, rnn_model, batch_size,
          labels, optimizer, epoch, ixtoword, image_dir, scaler):
    # dataloader : 데이터셋을 불러오는 DataLoader 객체
    # labels : 배치 크기만큼의 정수 라벨 텐서 (0 ~ batch_size -1)
    # ixtoword : 인덱스를 단어로 변환하는 딕셔너리
    # image__dir : 학습 중 생성된 attentio map 이미지를 저장할 디렉토리

    cnn_model.train() # 모델이 학습 모드로 설정됨 → Dropout과 같이 학습 중에만 활성화되는 연산이 실행됨됨
    rnn_model.train()
    s_total_loss0 = 0
    s_total_loss1 = 0
    w_total_loss0 = 0
    w_total_loss1 = 0
    count = (epoch + 1) * len(dataloader) # 학습된 데이터 샘플 개수를 추적
    start_time = time.time() # 학습 속도를 측정하기 위한 시작 시간 저장
    for step, data in enumerate(dataloader, 0):
        # dataloader에서 데이터를 하나씩 꺼내 학습을 진행

        # print('step', step)
        # 이전 step의 그래디언트 값을 초기화화
        rnn_model.zero_grad()
        cnn_model.zero_grad()

        imgs, captions, cap_lens, class_ids, keys = prepare_data(data)
        # prepare_data를 이용하여 데이터 정렬

        with autocast(device_type='cuda'):
            words_features, sent_code = cnn_model(imgs[-1]) # CNN 인코더 적용
            nef, att_sze = words_features.size(1), words_features.size(2)
            hidden = rnn_model.init_hidden(batch_size) # RNN 인코더 적용
            words_emb, sent_emb = rnn_model(captions, cap_lens, hidden)

            w_loss0, w_loss1, attn_maps = words_loss(words_features, words_emb, labels,
                                                 cap_lens, class_ids, batch_size)
            w_total_loss0 += w_loss0.item()
            w_total_loss1 += w_loss1.item()
            loss = w_loss0 + w_loss1

            s_loss0, s_loss1 = \
            sent_loss(sent_code, sent_emb, labels, class_ids, batch_size)
            loss += s_loss0 + s_loss1
            s_total_loss0 += s_loss0.item()
            s_total_loss1 += s_loss1.item()

        scaler.scale(loss).backward()  # loss에 대해 그래디언트를 계산하고 스케일링
        torch.nn.utils.clip_grad_norm_(rnn_model.parameters(), cfg.TRAIN.RNN_GRAD_CLIP)
        
            # optimizer.step() 전에 scaler.step()을 호출하여 가중치를 업데이트
        scaler.step(optimizer)  # 가중치 업데이트
        scaler.update()  # 스케일러 업데이트

        if step % UPDATE_INTERVAL == 0:
                # 일정한 간격마다 손실값 및 학습 속도 출력
            count = epoch * len(dataloader) + step

            s_cur_loss0 = s_total_loss0 / UPDATE_INTERVAL
            s_cur_loss1 = s_total_loss1 / UPDATE_INTERVAL

            w_cur_loss0 = w_total_loss0 / UPDATE_INTERVAL
            w_cur_loss1 = w_total_loss1 / UPDATE_INTERVAL

            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches | ms/batch {:5.2f} | '
                    's_loss {:5.2f} {:5.2f} | '
                    'w_loss {:5.2f} {:5.2f}'
                    .format(epoch, step, len(dataloader),
                            elapsed * 1000. / UPDATE_INTERVAL,
                            s_cur_loss0, s_cur_loss1,
                            w_cur_loss0, w_cur_loss1))
            s_total_loss0 = 0
            s_total_loss1 = 0
            w_total_loss0 = 0
            w_total_loss1 = 0
            start_time = time.time()
                # attention Maps
            img_set, _ = \
                    build_super_images(imgs[-1].cpu(), captions, ixtoword, attn_maps, att_sze)
            """
            build_super_images : 실제 이미지, 어텐션 맵, 캡션을 기반으로 시각적 주의를 시각화한 이미지를 생성하는 역할
            텍스트 설명과 함께 이미지와 attention map을 한 장의 이미지로 구성.
            img_set : attention map이 적용된 이미지가 저장됨
            _ : 두 번째 반환값이 있지만 사용되지 않음을 의미
            imgs[-1].cpu() : 마지막 단계의 이미지 텐서를 CPU로 이동시킴

            """

            # 이미지가 유효한 경우 실행
            if img_set is not None:
                im = Image.fromarray(img_set)
                # Numpy 배열을 PIL 이미지 객체로 변환
                image_dir_per_epoch = os.path.join(image_dir, str(epoch))
                mkdir_p(image_dir_per_epoch)
                fullpath = '%s/attention_maps_step%d.png' % (image_dir_per_epoch, step)
                # 저장 경로 설정, step : 현재 학습 단계(step) 번호
                im.save(fullpath)
    return count


def next_evaluate(dataloader, cnn_model, rnn_model, batch_size, labels):
    cnn_model.eval() # 모델이 평가 모드로 설정됨
    rnn_model.eval()
    s_total_loss = 0
    w_total_loss = 0

    with autocast(device_type='cuda'):
        for step, data in enumerate(dataloader, 0):
            real_imgs, captions, cap_lens, class_ids, keys = prepare_data(data)

            words_features, sent_code = cnn_model(real_imgs[-1])
            # nef = words_features.size(1)
            # words_features = words_features.view(batch_size, nef, -1)

            hidden = rnn_model.init_hidden(batch_size)
            words_emb, sent_emb = rnn_model(captions, cap_lens, hidden)

            w_loss0, w_loss1, attn = words_loss(words_features, words_emb, labels,
                                                cap_lens, class_ids, batch_size)
            w_total_loss += (w_loss0 + w_loss1).data

            s_loss0, s_loss1 = \
                sent_loss(sent_code, sent_emb, labels, class_ids, batch_size)
            s_total_loss += (s_loss0 + s_loss1).data

            if step == 50:
                break

    s_cur_loss = s_total_loss.item() / step
    w_cur_loss = w_total_loss.item() / step

    return s_cur_loss, w_cur_loss

def build_next_models(dataset, batch_size):
    # build model ############################################################
    text_encoder = RNN_ENCODER(dataset.n_words, nhidden=cfg.TEXT.EMBEDDING_DIM)
    # RNN_ENCODER : 텍스트 데이터를 인코딩하는 RNN 모델
    # n_words : 단어 크기 (텍스트에 포함된 전체 단어
    # cfg.TEXT.EMBEDDING_DIM = 256, RNN의 은닉 상태 크기

    image_encoder = CNN_ENCODER(cfg.TEXT.EMBEDDING_DIM)
    # CNN_ENCODER : 이미지를 인코딩하는 CNN 모델
    # 텍스 임베딩 차원을 입력으로 받음 → 나중에 텍스트와 이미지 특징을 비교하기 위해해

    # labels = Variable(torch.LongTensor(range(batch_size)))
    labels = torch.arange(batch_size, dtype=torch.long)
    # 배치 크기만큼의 정수 라벨 텐서를 생성 : [0, 1, 2, ..., batch_size -1]
    # 모델 학습 시, 배치 내 각 샘플의 인덱스를 나타내는 역할

    start_epoch = 0
    model_path = '../output/coco_DAMSM_2025_02_19_00_04_58/Model/text_encoder35.pth'
    state_dict = torch.load(model_path)
    text_encoder.load_state_dict(state_dict)
    print('Load ', model_path)
    

    name = '../output/coco_DAMSM_2025_02_19_00_04_58/Model/image_encoder35.pth'
    state_dict = torch.load(name)
    image_encoder.load_state_dict(state_dict)
    print('Load ', name)
    
    model_filename = model_path.split('/')[-1]
    epoch_str = re.search(r'\d+', model_filename).group()  # 첫 번째 숫자 추출
    start_epoch = int(epoch_str) + 1  # 다음 epoch 번호

    print('start_epoch', start_epoch)

    if cfg.CUDA:
        # cfg.CUDA == True라면, GPU에서 모델을 실행하도록 .cuda()를 호출
        text_encoder = text_encoder.cuda()
        image_encoder = image_encoder.cuda()
        labels = labels.cuda() # 배치별 정수 라벨 텐서
        
    return text_encoder, image_encoder, labels, start_epoch

image_dir ='../output/coco_DAMSM_2025_02_19_00_04_58/Image'
model_dir = '../output/coco_DAMSM_2025_02_19_00_04_58/Model'
text_encoder, image_encoder, labels, start_epoch = build_next_models(dataset_train, batch_size)
scaler = GradScaler()
optimizer = optim.Adam(para, lr=cfg.TRAIN.ENCODER_LR, betas=(0.5, 0.999))
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.98)

para = list(text_encoder.parameters())
# text_encoder의 모든 학습 가능 파라미터를 리스트에 추가
for v in image_encoder.parameters():
    if v.requires_grad:
        # requires_grad=False인 경우 역전파에서 제외시킴
        para.append(v)
# optimizer = optim.Adam(para, lr=cfg.TRAIN.ENCODER_LR, betas=(0.5, 0.999))
# At any point you can hit Ctrl + C to break out of training early.

# 가장 낮은 검증 손실을 저장
best_val_loss = float('inf')

# 옵티마이저를 한 번만 생성
try:
    lr = cfg.TRAIN.ENCODER_LR # 초기 학습률(lr) 설정
    for epoch in range(start_epoch, cfg.TRAIN.MAX_EPOCH):
        # start_epoch부터 MAX_EPOCH까지 학습 반복
        # optimizer = optim.Adam(para, lr=lr, betas=(0.5, 0.999))
        # Adam 옵티마이저를 사용하여 text/image_encoder를 학습
        # 일반적으로는 betas=(0.9, 0.999)를 이용

        epoch_start_time = time.time()
        count = next_train(dataloader_train, image_encoder, text_encoder,
                    batch_size, labels, optimizer, epoch,
                    dataset_train.ixtoword, image_dir, scaler)
        # 모델을 한 epoch 동안 학습하는 역할을 함
        print('-' * 89)

        if len(dataloader_val) > 0:
            # 학습한 모델을 검증 데이터에서 평가
            # 가중치를 업데이트하지 않고, 현재 모델이 얼마나 잘 동작하는지 평가가
            s_loss, w_loss = next_evaluate(dataloader_val, image_encoder,
                                      text_encoder, batch_size, labels)
            val_loss = s_loss + w_loss # 검증 손실 합산
            print('| end epoch {:3d} | valid loss {:5.2f} {:5.2f} | lr {:.5f}|'
                  .format(epoch, s_loss, w_loss, scheduler.get_last_lr()[0]))
            
            #### 가장 낮은 검증 손실을 가진 모델 저장
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                torch.save(image_encoder.state_dict(),
                           f'{model_dir}/best_image_encoder.pth')
                torch.save(text_encoder.state_dict(),
                           f'{model_dir}/best_text_encoder.pth')
                print('Best model saved!')
            
        print('-' * 89)
        
        scheduler.step() # 스케줄러로 학습률 감소

        #if lr > cfg.TRAIN.ENCODER_LR/10.:
            # 현재 학습률이 초기 학습률의 1/10 이상이면 lr를 98%로 감소시킴
        #    lr *= 0.98

        # 모델 저장
        if (epoch % cfg.TRAIN.SNAPSHOT_INTERVAL == 0 or
            epoch == cfg.TRAIN.MAX_EPOCH):
            # 일정 epoch마다 또는 MAX_EPOCH이 되면 가중치 저장
            torch.save(image_encoder.state_dict(),
                       '%s/image_encoder%d.pth' % (model_dir, epoch))
            torch.save(text_encoder.state_dict(),
                       '%s/text_encoder%d.pth' % (model_dir, epoch))
            print('Save models.')
            
except KeyboardInterrupt:
    print('-' * 89)
    print('Exiting from training early')

c:\Users\blues\anaconda3\envs\2024WINTERSIG\Lib\site-packages\torch\nn\modules\rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn(
c:\Users\blues\anaconda3\envs\2024WINTERSIG\Lib\site-packages\torchvision\models\inception.py:43: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn(


Load pretrained model from  https://download.pytorch.org/models/inception_v3_google-1a9a5a14.pth
Load  ../output/coco_DAMSM_2025_02_19_00_04_58/Model/text_encoder35.pth


C:\Users\blues\AppData\Local\Temp\ipykernel_16608\2422545814.py:160: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(model_path)
C:\Users\blues\AppData

Load  ../output/coco_DAMSM_2025_02_19_00_04_58/Model/image_encoder35.pth
start_epoch 36


c:\Users\blues\anaconda3\envs\2024WINTERSIG\Lib\site-packages\torch\nn\modules\module.py:1736: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


| epoch  36 |     0/ 2464 batches | ms/batch 49.69 | s_loss  0.00  0.00 | w_loss  0.00  0.00
| epoch  36 |   200/ 2464 batches | ms/batch 1194.45 | s_loss  0.69  0.72 | w_loss  0.54  0.57
| epoch  36 |   400/ 2464 batches | ms/batch 1263.32 | s_loss  0.68  0.70 | w_loss  0.54  0.56
| epoch  36 |   600/ 2464 batches | ms/batch 1228.29 | s_loss  0.70  0.73 | w_loss  0.57  0.58
| epoch  36 |   800/ 2464 batches | ms/batch 1239.61 | s_loss  0.70  0.73 | w_loss  0.56  0.57
| epoch  36 |  1000/ 2464 batches | ms/batch 1306.84 | s_loss  0.69  0.72 | w_loss  0.55  0.56
| epoch  36 |  1200/ 2464 batches | ms/batch 1343.51 | s_loss  0.68  0.71 | w_loss  0.55  0.56
| epoch  36 |  1400/ 2464 batches | ms/batch 1322.38 | s_loss  0.67  0.70 | w_loss  0.53  0.55
| epoch  36 |  1600/ 2464 batches | ms/batch 1429.39 | s_loss  0.68  0.71 | w_loss  0.54  0.56
| epoch  36 |  1800/ 2464 batches | ms/batch 1479.87 | s_loss  0.70  0.72 | w_loss  0.55  0.57
| epoch  36 |  2000/ 2464 batches | ms/batch 1548.76

c:\Users\blues\anaconda3\envs\2024WINTERSIG\Lib\site-packages\torch\optim\lr_scheduler.py:224: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn(


| epoch  37 |     0/ 2464 batches | ms/batch 45.21 | s_loss  0.00  0.00 | w_loss  0.00  0.00
| epoch  37 |   200/ 2464 batches | ms/batch 1201.41 | s_loss  0.68  0.71 | w_loss  0.53  0.55
| epoch  37 |   400/ 2464 batches | ms/batch 1185.24 | s_loss  0.69  0.71 | w_loss  0.55  0.57
| epoch  37 |   600/ 2464 batches | ms/batch 1197.56 | s_loss  0.68  0.71 | w_loss  0.54  0.55
| epoch  37 |   800/ 2464 batches | ms/batch 1233.85 | s_loss  0.70  0.72 | w_loss  0.55  0.57
| epoch  37 |  1000/ 2464 batches | ms/batch 1254.42 | s_loss  0.68  0.71 | w_loss  0.55  0.57
| epoch  37 |  1200/ 2464 batches | ms/batch 1301.68 | s_loss  0.68  0.71 | w_loss  0.54  0.56
| epoch  37 |  1400/ 2464 batches | ms/batch 1344.66 | s_loss  0.70  0.73 | w_loss  0.55  0.57
| epoch  37 |  1600/ 2464 batches | ms/batch 1397.34 | s_loss  0.68  0.70 | w_loss  0.53  0.54
| epoch  37 |  1800/ 2464 batches | ms/batch 1538.97 | s_loss  0.69  0.72 | w_loss  0.55  0.57
| epoch  37 |  2000/ 2464 batches | ms/batch 1653.87

DAMSM 모델은 54epoch에서 학습을 마쳤으며, 54epoch이 Best Model Saved이기 때문에 나중에 이어서 학습할 때 Best Model로 이어서 학습 시키기기

In [11]:
# 여기부턴 GAN 모델 학습 과정
sys.argv = ['AttnGAN_train.py', '--cfg', 'cfg/coco_atn2.yml', '--gpu', '0']
# coco_atn2.yml
args = parse_args()
print(args)

Namespace(cfg_file='cfg/coco_atn2.yml', gpu_id=0, data_dir='', manualSeed=None)


In [12]:
if args.cfg_file is not None:
     cfg_from_file(args.cfg_file)

if args.gpu_id == -1:
    cfg.CUDA = False
else:
    cfg.GPU_ID = args.gpu_id

if args.data_dir != '':
    cfg.DATA_DIR = args.data_dir
    # DATA_DIR: '../data/coco'
print('Using config:')
pprint.pprint(cfg)

Using config:
{'B_VALIDATION': False,
 'CONFIG_NAME': 'glu-gan2',
 'CUDA': True,
 'DATASET_NAME': 'coco',
 'DATA_DIR': '../data/coco',
 'GAN': {'B_ATTENTION': True,
         'B_DCGAN': False,
         'CONDITION_DIM': 100,
         'DF_DIM': 96,
         'GF_DIM': 48,
         'R_NUM': 3,
         'Z_DIM': 100},
 'GPU_ID': 0,
 'RNN_TYPE': 'LSTM',
 'TEXT': {'CAPTIONS_PER_IMAGE': 5, 'EMBEDDING_DIM': 256, 'WORDS_NUM': 12},
 'TRAIN': {'BATCH_SIZE': 14,
           'B_NET_D': True,
           'DISCRIMINATOR_LR': 0.0002,
           'ENCODER_LR': 0.002,
           'FLAG': True,
           'GENERATOR_LR': 0.0002,
           'MAX_EPOCH': 50,
           'NET_E': '../DAMSMencoders/best_text_encoder.pth',
           'NET_G': '',
           'RNN_GRAD_CLIP': 0.25,
           'SMOOTH': {'GAMMA1': 4.0,
                      'GAMMA2': 5.0,
                      'GAMMA3': 10.0,
                      'LAMBDA': 50.0},
           'SNAPSHOT_INTERVAL': 3},
 'TREE': {'BASE_SIZE': 299, 'BRANCH_NUM': 3},
 'WORKE

In [13]:
print('manualSeed:', args.manualSeed)  # manualSeed 값 출력
if not cfg.TRAIN.FLAG:
    args.manualSeed = 100
elif args.manualSeed is None: # TRAIN.FLAG = true
    args.manualSeed = random.randint(1, 10000)
    
print('Updated manualSeed:', args.manualSeed)
random.seed(args.manualSeed)
np.random.seed(args.manualSeed)
torch.manual_seed(args.manualSeed)
if cfg.CUDA:
    torch.cuda.manual_seed_all(args.manualSeed)

manualSeed: None
Updated manualSeed: 1164


In [16]:
now = datetime.datetime.now(dateutil.tz.tzlocal())
timestamp = now.strftime('%Y_%m_%d_%H_%M_%S')
output_dir = '../output/%s_%s_%s' % \
        (cfg.DATASET_NAME, cfg.CONFIG_NAME, timestamp)

torch.cuda.set_device(cfg.GPU_ID)
# 저장된 GPU ID로 GPU를 설정한다.
cudnn.benchmark = True
# cuDNN의 최적화 활성화화

split_dir, bshuffle = 'train', True
if not cfg.TRAIN.FLAG:
    # bshuffle = False
    split_dir = 'test'
    # GAN 모델에서는 Validation 데이터 셋을 사용하지 않는 것으로 보임 -> Train만 쓰자!

# Get data loader
image_transform = transforms.Compose([
    transforms.Resize(int(imsize * 76 / 64)),
    transforms.RandomCrop(imsize),
    transforms.RandomHorizontalFlip()])

#### train dataset Load 
dataset_GAN_train = TextDataset(cfg.DATA_DIR, vocadict_dir, 'train',
                          base_size=cfg.TREE.BASE_SIZE,
                          transform=image_transform)
assert dataset_GAN_train, "Invalid train Dataset"


GAN_dataloader_train = torch.utils.data.DataLoader(
        dataset_GAN_train, batch_size=cfg.TRAIN.BATCH_SIZE,
        drop_last=True, shuffle=bshuffle, num_workers=int(cfg.WORKERS))

Load pickle file from:  ../data/coco\voca_dictionary\captions.pickle


In [18]:
from trainer import condGANTrainer as trainer

def gen_example(wordtoix, algo):
    '''generate images from example sentences'''
    from nltk.tokenize import RegexpTokenizer
    filepath = '%s/example_filenames.txt' % (cfg.DATA_DIR)
    data_dic = {}
    with open(filepath, "r") as f:
        filenames = f.read().decode('utf8').split('\n')
        for name in filenames:
            if len(name) == 0:
                continue
            filepath = '%s/%s.txt' % (cfg.DATA_DIR, name)
            with open(filepath, "r") as f:
                print('Load from:', name)
                sentences = f.read().decode('utf8').split('\n')
                # a list of indices for a sentence
                captions = []
                cap_lens = []
                for sent in sentences:
                    if len(sent) == 0:
                        continue
                    sent = sent.replace("\ufffd\ufffd", " ")
                    tokenizer = RegexpTokenizer(r'\w+')
                    tokens = tokenizer.tokenize(sent.lower())
                    if len(tokens) == 0:
                        print('sent', sent)
                        continue

                    rev = []
                    for t in tokens:
                        t = t.encode('ascii', 'ignore').decode('ascii')
                        if len(t) > 0 and t in wordtoix:
                            rev.append(wordtoix[t])
                    captions.append(rev)
                    cap_lens.append(len(rev))
            max_len = np.max(cap_lens)

            sorted_indices = np.argsort(cap_lens)[::-1]
            cap_lens = np.asarray(cap_lens)
            cap_lens = cap_lens[sorted_indices]
            cap_array = np.zeros((len(captions), max_len), dtype='int64')
            for i in range(len(captions)):
                idx = sorted_indices[i]
                cap = captions[idx]
                c_len = len(cap)
                cap_array[i, :c_len] = cap
            key = name[(name.rfind('/') + 1):]
            data_dic[key] = [cap_array, cap_lens, sorted_indices]
    algo.gen_example(data_dic)


# Define models and go to train/evaluate
algo = trainer(output_dir, GAN_dataloader_train, dataset_GAN_train.n_words, dataset_GAN_train.ixtoword)

start_t = time.time()
if cfg.TRAIN.FLAG:
    algo.train()
else: # cfg.TRAIN.FLAG = FAlSE 인 경우
    # generate images from pre-extracted embeddings
    if cfg.B_VALIDATION:
        algo.sampling(split_dir)  # generate images for the whole valid dataset
    else: # test
        gen_example(dataset_GAN_train.wordtoix, algo)  # generate images for customized captions
end_t = time.time()
print('Total time for training:', end_t - start_t)

Load pretrained model from  https://download.pytorch.org/models/inception_v3_google-1a9a5a14.pth
Load image encoder from: ../DAMSMencoders/best_image_encoder.pth
Load text encoder from: ../DAMSMencoders/best_text_encoder.pth
# of netsD 3


AttributeError: '_MultiProcessingDataLoaderIter' object has no attribute 'next'